In [ ]:
%pip install giza-datasets

In [ ]:
%pip install plotly pandas nbformat numpy scikit-learn xgboost matplotlib seaborn hvplot

In [ ]:
import certifi
import os
os.environ['SSL-CERT_FILE'] = certifi.where()
from giza_datasets import DatasetsLoader


loader = DatasetsLoader()
df = loader.load("gora-competition-training")
# df = loader.load('yearn-individual-deposits')

In [ ]:
import polars as pl

In [ ]:
df.head()

In [ ]:
import plotly.express as px

fig = px.scatter(x=df["total_deposit"], y=df["total_borrow"],color=df["liquidated_color"])
fig.show()

In [ ]:
df["token_borrow_mode"].unique()

In [ ]:
df2 = df["count_borrow"].sum().group_by()

fig = px.bar(df, x='token_borrow_mode', y='count_borrow')
fig.show()

In [ ]:
df2 = df.select(pl.col(["token_borrow_mode", "count_borrow"]))
df2

In [ ]:
df.group_by("token_borrow_mode").count()

In [ ]:
df = df.with_columns(token_borrow_mode = df["token_borrow_mode"].str.to_lowercase())

In [ ]:
with pl.Config(tbl_rows=52):
    print(df["token_borrow_mode"].unique())

In [ ]:
with pl.Config(tbl_rows=52):
    print(df.group_by("token_borrow_mode").agg(
        pl.col("liquidated").sum()
    ))

In [ ]:
df["liquidated"].unique()

In [ ]:
df.filter(df["total_liquidation_to_total_borrow"] > 0)

In [ ]:
import plotly.express as px

fig = px.scatter(y=df["total_liquidation"], x=df["avg_borrow_amount"])
fig.show()

In [ ]:
fig = px.scatter(x=df["total_liquidation"], y=df["count_borrow"])
fig.show()

In [ ]:
df.shape[0]

In [ ]:
columns_to_use = [
    "total_borrow",
    "count_borrow",
    "avg_borrow_amount",
    "std_borrow_amount",
    "borrow_amount_cv",
    "total_repay",
    "count_repay",
    "avg_repay_amount",
    "std_repay_amount",
    "repay_amount_cv",
    "total_deposit",
    "count_deposit",
    "avg_deposit_amount",
    "std_deposit_amount",
    "deposit_amount_cv",
    "total_redeem",
    "count_redeem",
    "avg_redeem_amount",
    "std_redeem_amount",
    "redeem_amount_cv",
    "days_since_first_borrow",
    "net_outstanding",
    "int_paid",
    "net_deposits",
    "count_repays_to_count_borrows",
    "avg_repay_to_avg_borrow",
    "net_outstanding_to_total_borrowed",
    "net_outstanding_to_total_repaid",
    "count_redeems_to_count_deposits",
    "total_redeemed_to_total_deposits",
    "avg_redeem_to_avg_deposit",
    "net_deposits_to_total_deposits",
    "net_deposits_to_total_redeemed",
    "dex_total_sum_added",
    "dex_total_sum_removed",
    "dex_total_sum_swapped",
    "total_liquidation_to_total_borrow"
]

# Feature Importance

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import math

def evaluate_linear_regression(x_train, y_train, x_test, y_test):
    m = LinearRegression()
    m.fit(X=x_train.to_numpy(), y=y_train.to_numpy())
    y_pred = m.predict(x_test.to_numpy())
    
    mse = mean_squared_error(y_test.to_numpy(), y_pred)
    rmse = math.sqrt(mse)

    # print("Linear Regression RMSE:", rmse)

    # Feature Importance
    importance = m.coef_[0]  # Don't know why I needed to add a 0 here
    importance = [(columns_to_use[i], v) for i, v in enumerate(importance)]
    importance.sort(key=lambda x: -x[1])
    # for (k, v) in importance:
    #     print('Feature %s, Score: %.5f' % (k, v))

    return rmse, importance


def evaluate_decision_tree(x_train, y_train, x_test, y_test):
    m = DecisionTreeRegressor()
    m.fit(X=x_train.to_numpy(), y=y_train.to_numpy())
    y_pred = m.predict(x_test.to_numpy())

    mse = mean_squared_error(y_test.to_numpy(), y_pred)
    rmse = math.sqrt(mse)

    # print("Decision Tree RMSE:", rmse)    

    importance = m.feature_importances_
    importance = [(columns_to_use[i], v) for i, v in enumerate(importance)]
    importance.sort(key=lambda x: -x[1])
    # for (k, v) in importance:
    #     print('Feature %s, Score: %.5f' % (k, v))

    return rmse, importance


def evaluate_random_forest(x_train, y_train, x_test, y_test):
    m = RandomForestRegressor()
    m.fit(X=x_train.to_numpy(), y=y_train.to_numpy())

    y_pred = m.predict(x_test.to_numpy())

    mse = mean_squared_error(y_test.to_numpy(), y_pred)
    rmse = math.sqrt(mse)

    # print("RMSE:", rmse)

    importance = m.feature_importances_
    importance = [(columns_to_use[i], v) for i, v in enumerate(importance)]
    importance.sort(key=lambda x: -x[1])
    # for (k, v) in importance:
    #     print('Feature %s, Score: %.5f' % (k, v))

    return rmse, importance


def evaluate_xg_boost(x_train, y_train, x_test, y_test):
    m = XGBRegressor()
    m.fit(x_train.to_numpy(), y_train.to_numpy())
    y_pred = m.predict(x_test.to_numpy())

    mse = mean_squared_error(y_test.to_numpy(), y_pred)
    rmse = math.sqrt(mse)

    # print("RMSE:", rmse)

    importance = m.feature_importances_
    importance = [(columns_to_use[i], v) for i, v in enumerate(importance)]
    importance.sort(key=lambda x: -x[1])
    # for (k, v) in importance:
    #     print('Feature %s, Score: %.5f' % (k, v))

    return rmse, importance


def log_to_file(filename, title, rmse, importance):
    with open(filename, "a") as f:
        f.write(f"============={title}=============\n")
        f.write(f"RMSE: {rmse}\n")
        for (k, v) in importance:
            f.write("Feature %s, Score: %.5f\n" % (k, v))


def get_training_and_test_data(df):
    DF = df.select(columns_to_use)

    test_size = int(DF.shape[0] * .1)
    test_df, train_df = DF.head(test_size), DF.tail(-test_size)

    x_train = train_df.select([pl.all().exclude("total_liquidation_to_total_borrow")])
    y_train = train_df.select("total_liquidation_to_total_borrow")
    x_test = test_df.select([pl.all().exclude("total_liquidation_to_total_borrow")])
    y_test = test_df.select("total_liquidation_to_total_borrow")

    return x_train, y_train, x_test, y_test

In [ ]:
rmse, importance = evaluate_linear_regression(x_train, y_train, x_test, y_test)
log_to_file("basic.txt", "Linear Regression", rmse, importance)

rmse, importance = evaluate_decision_tree(x_train, y_train, x_test, y_test)
log_to_file("basic.txt", "Decision Tree", rmse, importance)

rmse, importance = evaluate_xg_boost(x_train, y_train, x_test, y_test)
log_to_file("basic.txt", "XGBoost", rmse, importance)

In [ ]:
# Check if columns have null value
DF.select(pl.all().is_null().sum())

# Permutation Feature Importance

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance

In [ ]:
m = KNeighborsRegressor()
m.fit(x_train.to_numpy(), y_train.to_numpy())

In [ ]:
y_pred = m.predict(x_test.to_numpy())

mse = mean_squared_error(y_test.to_numpy(), y_pred)
rmse = math.sqrt(mse)

print("RMSE:", rmse)

In [ ]:
results = permutation_importance(m, x_train.to_numpy(), y_train.to_numpy(), scoring='neg_mean_squared_error')

In [ ]:
importance = results.importances_mean
for i, v in enumerate(importance):
    print('Feature %0d, Score: %.5f' % (i, v))

# Improvements

## 1. DEX Feature importance was 0.
Check if just checking activity has any changes.

In [ ]:
df.select(["dex_total_sum_swapped", "dex_total_sum_added", "dex_total_sum_removed"]).unique()

## 2. Normalize total_* amounts by the total_deposit

In [ ]:
df.select(["total_borrow", "total_repay", "total_deposit", "total_redeem"]).head()

In [ ]:
df2 = df.with_columns(
    normalized_total_borrow = pl.col("total_borrow") / pl.col("total_borrow"),
    normalized_total_repay = pl.col("total_repay") / pl.col("total_borrow"),
    normalized_total_deposit = pl.col("total_deposit") / pl.col("total_borrow"),
    normalized_total_redeem = pl.col("total_redeem") / pl.col("total_borrow"),
)

In [ ]:
columns_to_use = [
    "normalized_total_borrow",
    "normalized_total_repay",
    "normalized_total_deposit",
    "normalized_total_redeem",
    "total_deposit",
    "count_borrow",
    "avg_borrow_amount",
    "std_borrow_amount",
    "borrow_amount_cv",
    "count_repay",
    "avg_repay_amount",
    "std_repay_amount",
    "repay_amount_cv",
    "count_deposit",
    "avg_deposit_amount",
    "std_deposit_amount",
    "deposit_amount_cv",
    "count_redeem",
    "avg_redeem_amount",
    "std_redeem_amount",
    "redeem_amount_cv",
    "days_since_first_borrow",
    "net_outstanding",
    "int_paid",
    "net_deposits",
    "count_repays_to_count_borrows",
    "avg_repay_to_avg_borrow",
    "net_outstanding_to_total_borrowed",
    "net_outstanding_to_total_repaid",
    "count_redeems_to_count_deposits",
    "total_redeemed_to_total_deposits",
    "avg_redeem_to_avg_deposit",
    "net_deposits_to_total_deposits",
    "net_deposits_to_total_redeemed",
    "total_liquidation_to_total_borrow"
]
x_train, y_train, x_test, y_test = get_training_and_test_data(df2)

In [ ]:
# df2.select(pl.all().is_null().sum())
df2.select([pl.col(pl.Int64), pl.col(pl.Int32), pl.col(pl.Float32), pl.col(pl.Float64), pl.col(pl.Decimal)]).select(pl.all().is_nan().sum())

In [ ]:
rmse, importance = evaluate_linear_regression(x_train, y_train, x_test, y_test)
log_to_file("normalized_totals.txt", "Linear Regression", rmse, importance)

rmse, importance = evaluate_decision_tree(x_train, y_train, x_test, y_test)
log_to_file("normalized_totals.txt", "Decision Tree", rmse, importance)

rmse, importance = evaluate_xg_boost(x_train, y_train, x_test, y_test)
log_to_file("normalized_totals.txt", "XGBoost", rmse, importance)

## 3. Avg borrow to Total Borrow

In [ ]:
df2 = df.with_columns(
    normalized_total_borrow = pl.col("total_borrow") / pl.col("total_borrow"),
    normalized_total_repay = pl.col("total_repay") / pl.col("total_borrow"),
    normalized_total_deposit = pl.col("total_deposit") / pl.col("total_borrow"),
    normalized_total_redeem = pl.col("total_redeem") / pl.col("total_borrow"),
    avg_to_total_borrow = pl.col("avg_borrow_amount") / pl.col("total_borrow"),
    avg_to_total_repay = pl.col("avg_repay_amount") / pl.col("total_repay"),
    avg_to_total_deposit = pl.col("avg_deposit_amount") / pl.col("total_deposit"),
    # avg_to_total_redeem = pl.col("avg_deposit_amount") / pl.col("total_redeem"),
)

In [ ]:
print(df2.select([pl.col(pl.Int64), pl.col(pl.Int32), pl.col(pl.Float32), pl.col(pl.Float64), pl.col(pl.Decimal)]).select(pl.all().is_infinite().sum()))

In [ ]:
df2 = df2.fill_nan(0)

In [ ]:
columns_to_use = [
    "normalized_total_borrow",
    "normalized_total_repay",
    "normalized_total_deposit",
    "normalized_total_redeem",
    "avg_to_total_borrow",
    "avg_to_total_repay",
    "avg_to_total_deposit",
    # "avg_to_total_redeem",
    "total_deposit",
    "count_borrow",
    "avg_borrow_amount",
    "std_borrow_amount",
    "borrow_amount_cv",
    "count_repay",
    "avg_repay_amount",
    "std_repay_amount",
    "repay_amount_cv",
    "count_deposit",
    "avg_deposit_amount",
    "std_deposit_amount",
    "deposit_amount_cv",
    "count_redeem",
    "avg_redeem_amount",
    "std_redeem_amount",
    "redeem_amount_cv",
    "days_since_first_borrow",
    "net_outstanding",
    "int_paid",
    "net_deposits",
    "count_repays_to_count_borrows",
    "avg_repay_to_avg_borrow",
    "net_outstanding_to_total_borrowed",
    "net_outstanding_to_total_repaid",
    "count_redeems_to_count_deposits",
    "total_redeemed_to_total_deposits",
    "avg_redeem_to_avg_deposit",
    "net_deposits_to_total_deposits",
    "net_deposits_to_total_redeemed",
    "total_liquidation_to_total_borrow"
]
x_train, y_train, x_test, y_test = get_training_and_test_data(df2)

rmse, importance = evaluate_linear_regression(x_train, y_train, x_test, y_test)
log_to_file("normalized_totals_avg_borrow.txt", "Linear Regression", rmse, importance)

rmse, importance = evaluate_decision_tree(x_train, y_train, x_test, y_test)
log_to_file("normalized_totals_avg_borrow.txt", "Decision Tree", rmse, importance)

rmse, importance = evaluate_xg_boost(x_train, y_train, x_test, y_test)
log_to_file("normalized_totals_avg_borrow.txt", "XGBoost", rmse, importance)

In [ ]:
df.select(pl.all()).describe()

In [ ]:
df2

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')

In [ ]:
# count_borrow_values = df['count_borrow'].value_counts().sort(by='count', descending=True)

# count_borrow_values = df['count_borrow'].value_counts().sort(by='count_borrow')

count_borrow_values = df.filter(pl.col("liquidated") == 1)['count_borrow'].value_counts().sort(by='count_borrow')

In [ ]:
count_borrow_values.to_pandas().plot(kind='bar', title="Borrow Counts Frequency", x='count_borrow', y='count', rot=90)

In [ ]:
count_borrow_values.plot(kind='bar', title="Borrow Counts Frequency", x='count_borrow', y='count', rot=90)

In [ ]:
fig = px.bar(count_borrow_values, x="count_borrow", y="count")
fig.show()

In [ ]:
fig = px.bar(x=count_borrow_values["count_borrow"], y=count_borrow_values["count"])
fig.show()

In [ ]:
count_borrow_values["count"]

In [ ]:
count_deposit_values = df.filter(pl.col("liquidated") == 1)['count_deposit'].value_counts().sort(by='count_deposit')
count_deposit_values.plot(kind='bar', title="Borrow Deposit Frequency", x='count_deposit', y='count', rot=90)

In [ ]:
count_deposit_values = df['count_deposit'].value_counts().sort(by='count_deposit')
count_deposit_values.plot(kind='bar', title="Borrow Deposit Frequency", x='count_deposit', y='count', rot=90)

In [ ]:
fig = px.scatter(df, x="count_deposit", y="count_borrow")
fig.show()

In [ ]:
fig = px.scatter(df.filter(pl.col("liquidated")==1), x="count_deposit", y="count_borrow")
fig.show()

In [ ]:
sns.scatterplot(data=df, x="count_deposit", y="count_borrow", hue="liquidated")

In [ ]:
sns.pairplot(df.to_pandas(), vars=["count_deposit", "count_borrow", "count_repay", "count_redeem"], hue="liquidated")

In [ ]:
sns.pairplot(df.to_pandas(), vars=["avg_deposit_amount", "avg_borrow_amount", "avg_repay_amount", "avg_redeem_amount"], hue="liquidated")

In [ ]:
sns.scatterplot(data=df, x="total_deposit", y="avg_borrow_amount", hue="liquidated")

In [ ]:
df2 = df.with_columns(
    normalized_total_borrow = pl.col("total_borrow") / pl.col("total_borrow"),
    normalized_total_repay = pl.col("total_repay") / pl.col("total_borrow"),
    normalized_total_deposit = pl.col("total_deposit") / pl.col("total_borrow"),
    normalized_total_redeem = pl.col("total_redeem") / pl.col("total_borrow"),
    total_deposit_by_avg_borrow_amount = pl.col("total_deposit") / pl.col("avg_borrow_amount")
)

In [ ]:
columns_to_use = [
    "normalized_total_borrow",
    "normalized_total_repay",
    "normalized_total_deposit",
    "normalized_total_redeem",
    "total_deposit_by_avg_borrow_amount",
    "total_deposit",
    "count_borrow",
    "avg_borrow_amount",
    "std_borrow_amount",
    "borrow_amount_cv",
    "count_repay",
    "avg_repay_amount",
    "std_repay_amount",
    "repay_amount_cv",
    "count_deposit",
    "avg_deposit_amount",
    "std_deposit_amount",
    "deposit_amount_cv",
    "count_redeem",
    "avg_redeem_amount",
    "std_redeem_amount",
    "redeem_amount_cv",
    "days_since_first_borrow",
    "net_outstanding",
    "int_paid",
    "net_deposits",
    "count_repays_to_count_borrows",
    "avg_repay_to_avg_borrow",
    "net_outstanding_to_total_borrowed",
    "net_outstanding_to_total_repaid",
    "count_redeems_to_count_deposits",
    "total_redeemed_to_total_deposits",
    "avg_redeem_to_avg_deposit",
    "net_deposits_to_total_deposits",
    "net_deposits_to_total_redeemed",
    "total_liquidation_to_total_borrow"
]
x_train, y_train, x_test, y_test = get_training_and_test_data(df2)

rmse, importance = evaluate_linear_regression(x_train, y_train, x_test, y_test)
log_to_file("deposit_avg_borrow.txt", "Linear Regression", rmse, importance)

rmse, importance = evaluate_decision_tree(x_train, y_train, x_test, y_test)
log_to_file("deposit_avg_borrow.txt", "Decision Tree", rmse, importance)

rmse, importance = evaluate_xg_boost(x_train, y_train, x_test, y_test)
log_to_file("deposit_avg_borrow.txt", "XGBoost", rmse, importance)

In [ ]:
sns.pairplot(df.to_pandas(), x_vars=["net_deposits", "net_outstanding"], y_vars=["avg_deposit_amount", "avg_borrow_amount", "avg_repay_amount", "avg_redeem_amount"], hue="liquidated")

In [ ]:
sns.pairplot(df.to_pandas(), x_vars=["net_deposits", "net_outstanding"], y_vars=["avg_deposit_amount", "avg_borrow_amount", "avg_repay_amount", "avg_redeem_amount"], hue="total_liquidation")

In [ ]:
sns.pairplot(df.to_pandas(), x_vars=["net_deposits", "net_outstanding"], y_vars=["avg_deposit_amount", "avg_borrow_amount", "avg_repay_amount", "avg_redeem_amount"], hue="total_liquidation_to_total_borrow")

In [ ]:
df["address"].shape

In [ ]:
df["address"].value_counts().sort(by='count', descending=True)["count"].plot(kind='hist', bins = 40)

In [ ]:
with pl.Config(fmt_str_lengths=1000):
    print(df["address"].value_counts().sort(by='count', descending=True).head())

In [ ]:
df.filter(pl.col("address") == "0x6298f9fd4ae8b64b577dcc56cc72d18f649e3961")

In [ ]:
df.unique(["address", "count_borrow"])

In [ ]:
df.unique(["address", "count_borrow", "total_liquidation_to_total_borrow"])

In [ ]:
with pl.Config(fmt_str_lengths=1000):
    print(df.unique(["address", "count_borrow", "total_liquidation_to_total_borrow"])["address"].value_counts().sort(by='count', descending=True).head())

In [ ]:
df.unique(["address", "count_borrow", "total_liquidation_to_total_borrow"]).filter(pl.col("address") == "0x7487f314ebc3a8ab7221a7130560c0734f92a4c6")

In [ ]:
df.unique(["address", "count_borrow", "total_liquidation_to_total_borrow"]).filter(pl.col("address") == "0x7487f314ebc3a8ab7221a7130560c0734f92a4c6").unique("liquidated")

In [ ]:
x_train["address"].set_intersection